## 1. SAIGE step1

In [ ]:
%%bash

job_directory=$PWD/.job
mkdir -p $job_directory
job_file="${job_directory}/case1_${group}.job"

for group in 1 2 3 #For three different subgroups
do
    for seed in {1..100}
    do
        for prev in 0.01 0.05
        do
            mkdir -p /home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step1_prev${prev}
            job_file="${job_directory}/case1_${group}.job"
            echo "#!/bin/bash
            #SBATCH --job-name=WB_case1_group${group}_step1
            #SBATCH --output=WB_case1_group${group}_step1.out
            #SBATCH --error=WB_case1_group${group}_step1.err
            #SBATCH --partition=leelabsg
            #SBATCH --ntasks=1
            #SBATCH --cpus-per-task=32
            srun Rscript /home/parkeunj/SAIGE/extdata/step1_fitNULLGLMM.R \
                --plinkFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_plink/wb_WES_ukb_case1_group${group}_oddchr \
                --phenoFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/whole_WES/seed_${seed}_WES_200k_WB_tau_1_prev_${prev}.pheno.oddChr_pca \
                --phenoCol=y \
                --covarColList=x1,x2,PC1,PC2,PC3,PC4 \
                --sampleIDColinphenoFile=IND_ID \
                --traitType=binary \
                --nThreads=64 \
                --isCateVarianceRatio=TRUE \
                --outputPrefix=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step1_prev${prev}/WB_case1_group${group}_step1_seed${seed} \
                --sparseGRMFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/White_British__relatednessCutoff_0.05_5000_randomMarkersUsed.sparseGRM.mtx \
                --sparseGRMSampleIDFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/White_British__relatednessCutoff_0.05_5000_randomMarkersUsed.sparseGRM.mtx.sampleIDs.txt \
                --IsOverwriteVarianceRatioFile=FALSE \
                --LOCO=FALSE" > $job_file
            sbatch --output=./.job/step1_${seed}_${chr} --cpus-per-task=32 --partition=leelabsg --mem=80G --exclude=leelabsg06 $job_file
            
        done
    done
done



## 2. SAIGE step2

In [ ]:
%%bash

job_directory=$PWD/.job
mkdir -p $job_directory
job_file="${job_directory}/case1_${group}.job"

for group in 1 2 3
do

    for prev in 0.01 0.05
    do
        for seed in {1..100}
        do
            for ((chr=2; chr<=22; chr=chr+2))
            do
                mkdir -p /home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step2_prev${prev}
                job_file="${job_directory}/case1_${group}.job"
                echo "#!/bin/bash
                #SBATCH --job-name=WB_case1_group${group}_step2
                #SBATCH --output=WB_case1_group${group}_step2.out
                #SBATCH --error=WB_case1_group${group}_step2.err
                #SBATCH --partition=leelabsg
                #SBATCH --ntasks=1
                #SBATCH --cpus-per-task=1
                srun Rscript /home/parkeunj/SAIGE/extdata/step2_SPAtests.R \
                    --bedFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_plink/wb_WES_ukb_case1_group${group}_chr${chr}.bed \
                    --bimFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_plink/wb_WES_ukb_case1_group${group}_chr${chr}.bim \
                    --famFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_plink/wb_WES_ukb_case1_group${group}_chr${chr}.fam \
                    --AlleleOrder=alt-first \
                    --SAIGEOutputFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step2_prev${prev}/WB_case1_group${group}_step2_seed${seed}_chr${chr}.txt \
                    --chrom=${chr} \
                    --minMAF=0 \
                    --minMAC=0.5 \
                    --is_overwrite_output=TRUE \
                    --GMMATmodelFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step1_prev${prev}/WB_case1_group${group}_step1_seed${seed}.rda \
                    --varianceRatioFile=/home/leelabsg/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step1_prev${prev}/WB_case1_group${group}_step1_seed${seed}.varianceRatio.txt \
                    --maxMAF=0.01 \
                    --is_Firth_beta=TRUE \
                    --pCutoffforFirth=0.05 \
                    --is_output_moreDetails=TRUE \
                    --max_MAC_for_ER=10 \
                    --LOCO=FALSE" > $job_file
                sbatch --output=./.job/${group}_${seed}_${chr} --cpus-per-task=1 --partition=leelabsg --exclude=leelabsg[07,11] --mem=15G $job_file
            done
            
        done
    done
done


## 3. SAIGE step3 (LD Matrix Generation)

In [ ]:
%%bash

for anno in lof missense_lof missense_lof_synonymous
do
    for maf in 0.01 0.001 0.0001
    do
        mkdir -p /media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step3_docker/${maf}_${anno}
        mkdir -p /media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step3_docker_log/${maf}_${anno}
        for chr in {1..22}
        do
            if [ $anno == "lof" ]
            then
                annotation_in_groupTest='lof'
            elif [ $anno == "missense_lof" ]
            then
                annotation_in_groupTest='missense;lof'
            elif [ $anno == "missense_lof_synonymous" ]
            then
                annotation_in_groupTest='missense;lof;synonymous'
            fi
            docker run -v /media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_plink:/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_plink \
            -v /media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step3_docker:/media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step3_docker \
            -v /media/leelabsg-storage0/eunjae/Meta-code/SAIGE_test:/media/leelabsg-storage0/eunjae/Meta-code/SAIGE_test \
            -w /media/leelabsg-storage0/eunjae/ wzhou88/saige:1.1.6.3 step3_LDmat.R \
                --bedFile=Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_plink/wb_WES_ukb_case1_group${group}_chr${chr}.bed \
                --bimFile=Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_plink/wb_WES_ukb_case1_group${group}_chr${chr}.bim \
                --famFile=Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_plink/wb_WES_ukb_case1_group${group}_chr${chr}.fam \
                --AlleleOrder=alt-first \
                --SAIGEOutputFile=Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step3_docker/${maf}_${anno}/WB_case1_group${group}_chr${chr} \
                --chrom=${chr} \
                --groupFile=Meta-code/SAIGE_test/groupfiles/UKBexomeOQFE_chr"$chr".gene.anno.hg38_PlinkMatch_v2.txt \
                --annotation_in_groupTest=${annotation_in_groupTest} \
                --is_overwrite_output=TRUE \
                --maxMAF_in_groupTest=${maf} \
                > /media/leelabsg-storage0/eunjae/Meta-code/Inflation_eval/WB_case1/group${group}/group${group}_step3_docker_log/${maf}_${anno}/WB_case1_group${group}_chr${chr}.log &
        done
    done
done


## 4. Meta-Analysis on DNANexus (Meta-SAIGE)

In [ ]:
%%bash

source ~/anaconda3/bin/activate dxpy # loading the conda environment from dnanexus python API
instance_type=mem2_ssd2_v2_x4
anno=missense_lof
maxMAF=0.01

for prev in 0.01 0.05
do

    for case in 1 2
    do
        dx mkdir -p UKB_Main:/Meta-SAIGE/WB_case${case}/meta_res_WBonly_V2_GC_prev${prev}_MaxMAF${maxMAF}
        destination=/Meta-SAIGE/WB_case${case}/meta_res_WBonly_V2_GC_prev${prev}_MaxMAF${maxMAF}
        for seed in {1..60}
        do
            for chr in {1..22}
            do
                dx run swiss-army-knife \
                -iin=UKB_Main:/Meta-SAIGE/WB_case${case}/group1/group1_step2_prev${prev}/WB_case${case}_group1_step2_seed${seed}_chr${chr}.txt \
                -iin=UKB_Main:/Meta-SAIGE/WB_case${case}/group2/group2_step2_prev${prev}/WB_case${case}_group2_step2_seed${seed}_chr${chr}.txt \
                -iin=UKB_Main:/Meta-SAIGE/WB_case${case}/group3/group3_step2_prev${prev}/WB_case${case}_group3_step2_seed${seed}_chr${chr}.txt \
                -iin=UKB_Main:/Meta-SAIGE/WB_case${case}/group1/group1_step3_docker/group1_${maxMAF}_${anno}.zip \
                -iin=UKB_Main:/Meta-SAIGE/WB_case${case}/group2/group2_step3_docker/group2_${maxMAF}_${anno}.zip \
                -iin=UKB_Main:/Meta-SAIGE/WB_case${case}/group3/group3_step3_docker/group3_${maxMAF}_${anno}.zip \
                -icmd="unzip -qq group1_${maxMAF}_${anno}.zip ; unzip -qq group2_${maxMAF}_${anno}.zip ; unzip -qq group3_${maxMAF}_${anno}.zip ; \
                    Rscript /app/RV_meta_GC.R \
                    --num_cohorts 3 \
                    --chr ${chr} \
                    --col_co 10 \
                    --info_file_path group1_${maxMAF}_${anno}/WB_case${case}_group1_chr${chr}.marker_info.txt \
                    group2_${maxMAF}_${anno}/WB_case${case}_group2_chr${chr}.marker_info.txt \
                    group3_${maxMAF}_${anno}/WB_case${case}_group3_chr${chr}.marker_info.txt \
                    \
                    --gene_file_prefix group1_${maxMAF}_${anno}/WB_case${case}_group1_chr${chr}_ \
                    group2_${maxMAF}_${anno}/WB_case${case}_group2_chr${chr}_ \
                    group3_${maxMAF}_${anno}/WB_case${case}_group3_chr${chr}_ \
                    \
                    --gwas_path WB_case${case}_group1_step2_seed${seed}_chr${chr}.txt \
                    WB_case${case}_group2_step2_seed${seed}_chr${chr}.txt \
                    WB_case${case}_group3_step2_seed${seed}_chr${chr}.txt \
                    \
                    --output_prefix WBonly_prev${prev}_case${case}_seed${seed}_chr${chr}_meta_res_MaxMAF${maxMAF}.txt ; \
                    rm -rf group1_${maxMAF}_${anno} ; rm -rf group2_${maxMAF}_${anno} ; rm -rf group3_${maxMAF}_${anno} ;" \
                -iimage_file=/docker_images/meta-saige.tar.gz \
                --destination ${destination} \
                --instance-type ${instance_type} \
                --ignore-reuse \
                --name=Meta-SAIGE_GC_prev${prev}_case${case}_seed${seed}_chr${chr} \
                --yes
            done
        done
    done

done